In [ ]:
# Import Libraries

import gc
import pickle
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
import os



In [ ]:

# Retrieve dataset and split into test and training sets
root = "/content/drive/MyDrive/DTSA 5511 - Deep Learning/esco_predict"
dir = os.path.join(root, "processed_data")
file = os.path.join(dir, "processed_training_set.pkl")
train_data = pd.read_pickle(file)

In [ ]:
# Separate features and traget variable
target = "usage_kWh"
X_train = train_data.drop(columns=[target])  # Drop target
y_train = train_data[target]

In [ ]:
# Create custom scorer for RMSLE
# Good article on RMSLE: https://medium.com/analytics-vidhya/root-mean-square-log-error-rmse-vs-rmlse-935c6cc1802a

def rmsle(y_true, y_pred):
  y_pred = np.clip(y_pred, 0, None) #Any value in y_pred that is less than 0 will be replaced with 0
  return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

In [ ]:
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from lightgbm import LGBMRegressor
from sklearn.metrics import make_scorer

def gridsearch_lgbm(X_train, y_train, param_grid):


  # Initialize TimeSeriesSplit
  tscv = TimeSeriesSplit(n_splits=5)

  rmsle_scorer = make_scorer(rmsle, greater_is_better=False)
  # mape = mean_absolute_percentage_error(y_test, y_pred)

  # Initialize the LGBM Regressor
  lgbm = LGBMRegressor(objective="regression", boosting_type="gbdt", metric="rmse")

  # Initialize GridSearchCV
  grid_search = GridSearchCV(
      estimator=lgbm,
      param_grid=param_grid,
      scoring=rmsle_scorer,
      cv=tscv,
      # verbose=0
  )

  # Perform grid search
  grid_search.fit(X_train, y_train)

  # Get the best parameters and the best score
  best_params = grid_search.best_params_
  best_score = -grid_search.best_score_  # Negate because RMSE scorer is reversed
  # print("Best Parameters:", best_params)
  print(f"Best RMSLE: {best_score}")
  return best_params



In [ ]:
#Free memory
# del train_df
# del test_df
# gc.collect() #Release memory
# print(best_params)

# Define LightGBM Parameters
def feature_importance(best_params, gain_tol, train_data, X_train, y_train):
    # feature_importance(best_params, gain_tol, X_train[imp_features], y_train)
  # best_params = {'learning_rate': 0.01, 'max_depth': 5, 'min_data_in_leaf': 50, 'num_leaves': 31, 'verbosity': -1}
  parameters = {
      "objective": "regression",
      "boosting_type": "gbdt",
      "seed": 42,
      "verbosity": -1,  # Suppress warnings and logs
  }

  parameters.update(best_params)

  # Train the model
  train_data = lgb.Dataset(X_train, label = y_train, params={"feature_pre_filter": False})
  lgb_model = lgb.train(parameters, train_data)

  # Make predictions on the test set
  y_pred = lgb_model.predict(X_train)

  importance = lgb_model.feature_importance(importance_type = "gain")
  feature_names = X_train.columns
  feature_importance_df = pd.DataFrame({"Feature": feature_names, "Importance": importance})
  feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)
  feature_importance_df["% Total Gain"] = round(feature_importance_df["Importance"] / feature_importance_df["Importance"].sum()*100, 2)
  print(feature_importance_df,"\n")

  imp_features_df = feature_importance_df[feature_importance_df["% Total Gain"] > 1.0]
  imp_features = imp_features_df["Feature"].tolist()
  # print(imp_features_df)
  X_train = X_train[imp_features]
  return X_train, imp_features_df

In [ ]:
def RFECV_lgbm(X_train, y_train, param_grid, gain_tol=0.01):
    """
    Recursive Feature Elimination with Cross-Validation for LightGBM

    Args:
        X_train (pd.DataFrame): Feature set for training
        y_train (pd.Series): Target variable for training
        param_grid (dict): Parameter grid for LightGBM model tuning
        gain_tol (float): Minimum percentage of total gain required for a feature to remain

    Returns:
        tuple: A tuple containing the important features and the best parameters
    """
    imp_features = X_train.columns.tolist()  # Start with all features
    best_params = {}
    iteration = 1

    while True:
        print(f"\nIteration {iteration}...")
        print(f"Using features: {imp_features}\n")

        # Grid search to find the best parameters
        best_params = gridsearch_lgbm(X_train[imp_features], y_train, param_grid)

        # Train the model and compute feature importance
        _, feature_importance_df = feature_importance(
            best_params, gain_tol, None, X_train[imp_features], y_train)

        # Identify features above the gain tolerance threshold
        current_imp_features = feature_importance_df[
            feature_importance_df["% Total Gain"] >= gain_tol * 100]["Feature"].tolist()

        print(f"Selected {len(current_imp_features)} features after filtering with gain_tol = {gain_tol * 100:.2f}%.")

        # Stop if no features were removed or only one feature is left
        if len(current_imp_features) == len(imp_features) or len(current_imp_features) <= 1:
            print(f"Stopping after iteration {iteration}. Remaining features: {current_imp_features}")
            break

        # Update features for the next iteration
        imp_features = current_imp_features
        iteration += 1

    return imp_features, best_params


## Parameter Descriptions for LightGBM

**learning_rate**:  Controls the step size for updating weights in each boosting iteration. Lower values make training slower but can improve accuracy by refining each step.

**num_iterations**:  *This code doesn't support the cross-validation of num_iterations. Uses default of 100.*  
The maximum number of boosting iterations (trees) to train. Higher values allow the model to learn more but increase training time.

**num_leaves**:  The maximum number of leaves in a single tree. A higher value allows more complex tree structures but increases the risk of overfitting.

**max_depth**:  The maximum depth of each tree. `-1` means no limit on tree depth. Constraining depth can reduce overfitting but may underfit the data if too shallow.

**min_data_in_leaf**:  Minimum number of samples required in a leaf node. Helps control overfitting; higher values create more generalized trees.

**verbosity**:  Controls the amount of output during training. `-1` means silent mode, useful for suppressing logs in automated workflows.


In [ ]:
#Define the parameter grid
# param_grid = {
#     "learning_rate": [0.01, 0.05, 0.1],
#     "num_leaves": [2, 5, 15, 20],
#     "max_depth": [2, 5, 10],
#     "min_data_in_leaf": [10, 20, 50, 100],
#     "metric": ['rmse', 'mae', 'mape'],
#     "verbosity": [-1]
# }

param_grid = {'learning_rate': [0.05], 'max_depth': [5], 'metric': ['rmse'], 'min_data_in_leaf': [50], 'num_leaves': [20], 'verbosity': [-1]}


imp_features, best_params = RFECV_lgbm(X_train, y_train, param_grid, gain_tol=0.01)

##Train the Model with Optimal Parameters
Also export the optimal parameters and features

In [ ]:
train_set = lgb.Dataset(X_train[imp_features], label = y_train)
final_model = lgb.train(params = best_params, train_set = train_set)

# Save the trained model to a .pkl file
import pickle
output_path = "/content/drive/MyDrive/DTSA 5511 - Deep Learning/esco_predict/trained_model/trained_model.pkl"
with open(output_path, "wb") as file:
    pickle.dump(final_model, file)

In [ ]:
print(imp_features)
print(best_params)

# Save both variables in a .pkl file
output_path = "/content/drive/MyDrive/DTSA 5511 - Deep Learning/esco_predict/trained_model/model_metadata.pkl"
with open(output_path, "wb") as pkl_file:
    pickle.dump({"imp_features": imp_features, "best_params": best_params}, pkl_file)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming your dataset is a Pandas DataFrame called `dataset`
# and your target variable is stored in the column 'usage_kWh'

# Define the features and target
X = X_train[imp_features]
y = y_train


# XGBoost parameters similar to the LGBM params
xgb_params = {
    'learning_rate': 0.05,
    'max_depth': 5,
    'objective': 'reg:squarederror',
    'n_estimators': 1000,  # Can be adjusted
    'min_child_weight': 50,  # Similar to min_data_in_leaf
    'colsample_bytree': 1.0,
    'subsample': 1.0,
    'verbosity': 0,
}

# Initialize the model
xgb_model = xgb.XGBRegressor(**xgb_params)

# Train the model without cross-validation or evaluation
xgb_model.fit(X_train, y_train, verbose=False)


output_path = "/content/drive/MyDrive/DTSA 5511 - Deep Learning/esco_predict/trained_model/xg_trained_model.pkl"
with open(output_path, "wb") as file:
    pickle.dump(final_model, file)